In [1]:
def read_dataframe(filename):
    import pandas as pd
    df= pd.read_parquet(filename)
    df['duration']=df['tpep_dropoff_datetime'] -df['tpep_pickup_datetime']
    df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)
    df['duration']=df['duration'].dt.total_seconds()/60
    df=df[df['payment_type']==1]
    categorical_columns=['PULocationID' , 'DOLocationID']
    df['location'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    for column in categorical_columns:
        df[column] = df[column].astype('category')
    return df

In [7]:
df_train  = read_dataframe('data/yellow_tripdata_2024-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2024-02.parquet')

In [ ]:
##
from sklearn.feature_extraction import DictVectorizer

categorical_columns=['location']
target = 'duration'
numerical = ['trip_distance']
y_train =df_train[target].values
y_val =df_val[target].values

dv= DictVectorizer()
train_dict = df_train[categorical_columns+numerical ].to_dict(orient='records')
X_train=dv.fit_transform(train_dict)


val_dicts = df_val[categorical_columns + numerical].to_dict(orient='records')
X_val =dv.transform(val_dicts)

In [ ]:
X_train

In [ ]:
X_val

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)


In [ ]:
y_pred=lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
X_val

In [ ]:
y_pred=lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
import pickle

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
## pytorck

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assume train_dict and df[target] are already defined and loaded as shown in your example
X_train = dv.fit_transform(train_dict).toarray()
X_val = dv.fit_transform(X_val).toarray()
y_train = df_train[target].values
y_val = df_val[target].values

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)


X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)


# Create DataLoader for batch processing
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)


In [ ]:
import torch.nn as nn

class TaxiTripModel(nn.Module):
    def __init__(self):
        super(TaxiTripModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = TaxiTripModel()


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            preds = model(X_batch)
            loss = loss_fn(preds, y_batch.unsqueeze(1))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_loss = sum(loss_fn(model(X_val), y_val.unsqueeze(1)) for X_val, y_val in val_loader) / len(val_loader)
        
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10)


In [ ]:
with open('models/TaxiTripModel.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)